# Restart
데이터셋 분류를 잘못했다. <br/>

test에 포함된 폰트의 글자가 train, val에도 섞여들었음을 확인했다. <br/>
다시 데이터셋을 만들텐데, 이번에는 test에 사용할 폰트를 미리 빼 놓자. 

---

또한 test셋을 출력할 때에는 같은 폰트(카테고리 말고!)끼리 묶여서 출력하도록 sorting하자.<br/>
- a\~Z 이미지를 만들어내기 전에, 폰트별로 먼저 split하고 a\~Z를 만들자.


---

In [9]:
import os
import glob
import shutil
from tqdm import tqdm
from PIL import Image,ImageDraw,ImageFont
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# 현재 폰트 위치 확인하기
path = '/Users/youngerous/Desktop/git repos/FontStyler/src/data/collection/selected/'
os.listdir(path)

['Monospace',
 'Serif',
 '.DS_Store',
 'Handwriting',
 'Sans Serif',
 '.gitignore',
 '.ipynb_checkpoints',
 'Display']

In [15]:
categories = ['Serif', 'Sans Serif', 'Display', 'Handwriting', 'Monospace']
dict_font = {
    'Serif': [],
    'Sans Serif': [],
    'Display': [],
    'Handwriting': [],
    'Monospace': []
}

for category in categories:
    fonts = path + category + '/'
    for font in glob.glob(fonts+'*.ttf'):
        dict_font[category].append(font)

In [16]:
for category in categories:
    print(len(dict_font[category]))

178
259
285
140
19


기존 폰트 개수랑 약간 차이가 나는데 .ttf가 아닌 .otf 파일도 섞여 있어서 그런 듯 하다. <br/>

큰 차이가 나지 않으니 무시하자.

---

## ttf 파일 split